# Preprocessing of data

### Import libraries and preproc

In [4]:
import sys
import numpy as np
from tqdm import tqdm
import nibabel
#!{sys.executable} -m pip install nilearn #install relevant neuroimaging package
#!{sys.executable} -m pip install opencv-python #install video package
from preprocessing import load_and_mask_miyawaki_data
from plots import plt_fmri_stim
import matplotlib.pyplot as plt

In [5]:
fmri_data, stimuli, fmri_figures_data, stimuli_figures, masker = load_and_mask_miyawaki_data()

Preprocessing data...

The shape of the masked data is (20, 145, 5438)
Preprocessed fMRI data: 2860 samples x 5438 voxels
Preprocessed stimuli data: 2860 samples x 100 pixels
1536 geometrical figures


 Done (21.20s).


### Create data sets

In [6]:
all_fmri = np.concatenate((fmri_data, fmri_figures_data))
all_stimuli = np.concatenate((stimuli, stimuli_figures))

#erase black figs: """"""useless"""""" for our use of the data
        
stimuli2 = np.array([element for element in stimuli if np.sum(element) != 0])
fmri_data2 = np.array([fmri_data[i] for i in range(len(fmri_data)) if np.sum(stimuli[i]) != 0])

all_stimuli2 = np.array([element for element in all_stimuli if np.sum(element) != 0])
all_fmri2 = np.array([all_fmri[i] for i in range(len(all_fmri)) if np.sum(all_stimuli[i]) != 0])

stimuli_figures2 = np.concatenate((np.array([element for element in stimuli_figures if np.sum(element) != 0]),np.array([stimuli_figures[4]])))
fmri_figures_data2 = np.concatenate((np.array([fmri_figures_data[i] for i in range(len(fmri_figures_data)) if np.sum(stimuli_figures[i]) != 0]), np.array([fmri_figures_data[4]])))

In [7]:
from sklearn.model_selection import train_test_split

#random images avec les carrés noirs
train_to_test_ratio = 0.8
X_train, X_test, Y_train, Y_test = train_test_split(fmri_data, stimuli, 
                                        train_size=train_to_test_ratio, 
                                        test_size=1.0-train_to_test_ratio)

#geometric images avec les carrés noirs
train_to_test_ratio = 0.8
X_trainfig, X_testfig, Y_trainfig, Y_testfig = train_test_split(fmri_figures_data, stimuli_figures, 
                                        train_size=train_to_test_ratio, 
                                        test_size=1.0-train_to_test_ratio)

#geometric images sans les carrés noirs
train_to_test_ratio = 0.8
X_trainfig2, X_testfig2, Y_trainfig2, Y_testfig2 = train_test_split(fmri_figures_data2, stimuli_figures2, 
                                        train_size=train_to_test_ratio, 
                                        test_size=1.0-train_to_test_ratio)

#random images sans les carrés noirs
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(fmri_data2, stimuli2, 
                                        train_size=train_to_test_ratio, 
                                        test_size=1.0-train_to_test_ratio)

#avec tout
X_train_all, X_test_all, Y_train_all, Y_test_all = train_test_split(all_fmri, all_stimuli, 
                                        train_size=train_to_test_ratio, 
                                        test_size=1.0-train_to_test_ratio)

X_train_all2, X_test_all2, Y_train_all2, Y_test_all2 =train_test_split(all_fmri2, all_stimuli2, train_size=train_to_test_ratio, test_size=1.0-train_to_test_ratio)

### Manipulate stimuli images for multiscale analysis

In [8]:
# Create list pixel_indices of 4-uplets of incidices of pixels we want to average together for the 2*2 scale
n_even = [2*i for i in range(5)] #list used to generate pixel_indices
#pixel_indices_22 = [[10*a+b, 10*a+(b+1), 10*(a+1)+b, 10*(a+1)+(b+1)] for a in n_even for b in n_even]
pixel_indices_22 = [[10*a+b, 10*a+(b+1), 10*(a+1)+b, 10*(a+1)+(b+1)] for a in range(9) for b in range(9)]
print(pixel_indices_22)

# Same for 2*1 scale
n_odd = [2*i+1 for i in range(50)]
pixel_indices_21 = [[a-1, a] for a in n_odd]

# Same for 1*2 scale
pixel_indices_12 = [[a*10 + b, (a+1)*10 +b] for a in n_even for b in range(10)]

[[0, 1, 10, 11], [1, 2, 11, 12], [2, 3, 12, 13], [3, 4, 13, 14], [4, 5, 14, 15], [5, 6, 15, 16], [6, 7, 16, 17], [7, 8, 17, 18], [8, 9, 18, 19], [10, 11, 20, 21], [11, 12, 21, 22], [12, 13, 22, 23], [13, 14, 23, 24], [14, 15, 24, 25], [15, 16, 25, 26], [16, 17, 26, 27], [17, 18, 27, 28], [18, 19, 28, 29], [20, 21, 30, 31], [21, 22, 31, 32], [22, 23, 32, 33], [23, 24, 33, 34], [24, 25, 34, 35], [25, 26, 35, 36], [26, 27, 36, 37], [27, 28, 37, 38], [28, 29, 38, 39], [30, 31, 40, 41], [31, 32, 41, 42], [32, 33, 42, 43], [33, 34, 43, 44], [34, 35, 44, 45], [35, 36, 45, 46], [36, 37, 46, 47], [37, 38, 47, 48], [38, 39, 48, 49], [40, 41, 50, 51], [41, 42, 51, 52], [42, 43, 52, 53], [43, 44, 53, 54], [44, 45, 54, 55], [45, 46, 55, 56], [46, 47, 56, 57], [47, 48, 57, 58], [48, 49, 58, 59], [50, 51, 60, 61], [51, 52, 61, 62], [52, 53, 62, 63], [53, 54, 63, 64], [54, 55, 64, 65], [55, 56, 65, 66], [56, 57, 66, 67], [57, 58, 67, 68], [58, 59, 68, 69], [60, 61, 70, 71], [61, 62, 71, 72], [62, 63, 

In [11]:
# Preprocessing of stimuli data for multi scale encoding

def preproc_one_stimuli(idx_pxl_toavg, stim, reshape = False, reshape_form = (5,5)) :
    """
    Permits to preproc 1 stimuli figure by averaging some pixels together for the 
    multiscale encoding.
    - idx_pxl_toavg : liste de multiplets d'indices de pixels que l'on souhaite moyenner
                      ensemble. Par ex, pour le 2*2 on a :
                      [[0,1,10,11], [2,3,12,13], ... ,[88,89,98,99]].
    - stimuli: the image
    - reshape : if you want your image to be reshaped in such a way that you can plot it
    - reshape_form : (5,5) for a 5 by 5 image, etc...
    """
    if reshape == False :
        return np.array([np.average([stim[i] for i in idx]) for idx in idx_pxl_toavg])
    
    else :
        return np.array(np.reshape([np.average([stim[i] for i in idx]) for idx in idx_pxl_toavg],reshape_form))
    

def preproc_multiple_stimuli(idx_pxl_toavg, stimuli_list, reshape = False, reshape_form = (5,5)) :
    """
    Same as above but to preproc a whole list of stimulis
    Sorry this looks horrible
    """
    if reshape == False :
        return np.array([[np.average([stim[i] for i in idx]) for idx in idx_pxl_toavg] for stim in stimuli_list])
    
    else :
        reshape_form_multi = (len(stimuli_list),reshape_form[0],reshape_form[1])
        return np.array(np.reshape([np.average([stim[i] for i in idx]) for idx in idx_pxl_toavg for stim in stimuli_list],reshape_form_multi))
    
def Y_train_newscale(Y_train, idx_pxl_toavg, reshape = False, reshape_form = (5,5)) :
    """
    Creates a new Y_train but with rescaled images 
    Changes greyscale values to classes (integers) : for this, we want to map the values of greyscale to
    integers. We notice that for 4 pixels, only possible values are [0,0.25,0.5,0.75,1]. For 2 pixels, [0,0.5,1]. Etc.
    By multiplying these grayscale values by the number of pixels we get [0,1,2,3,4] or [0,1,2], etc. Hence the multiplication.
    """
    Y_continuous = preproc_multiple_stimuli(idx_pxl_toavg, Y_train, reshape, reshape_form) #with values in [0,1]
    n = len(idx_pxl_toavg[0]) #number of classes is the number of pixels there is to average + 1. For instance for 4 pixels you can have [0,0.25,0.5,0.75,1]
    Y_integers = [[int(pixel_val * n) for pixel_val in img] for img in Y_continuous]
    return Y_integers

In [12]:
# Check that stimuli processing works

imag = stimuli_figures2[30]

old = np.reshape(imag,(10,10))
new = preproc_one_stimuli(pixel_indices_22, imag, reshape = True)

fig = plt.figure()
sp1 = plt.subplot(121)
sp2 = plt.subplot(122)

# to show the grid 

for ax in (sp1, sp2) :

    # Major ticks
    ax.set_xticks(np.arange(0, 10, 1))
    ax.set_yticks(np.arange(0, 10, 1))

    # Minor ticks
    ax.set_xticks(np.arange(-.5, 10, 1), minor=True)
    ax.set_yticks(np.arange(-.5, 10, 1), minor=True)
    
    ax.grid(which='minor', color='r', linestyle='-', linewidth=2)


sp1.imshow(old, cmap = plt.cm.gray, interpolation = 'nearest')
sp2.imshow(new, cmap = plt.cm.gray, interpolation = 'nearest')

ValueError: cannot reshape array of size 81 into shape (5,5)

It works !

# Train a log reg at the different scales

In [13]:
from sklearn import linear_model

def train_logreg_scale(X_train, Y_train, X_test, Y_test, scale) :
    """
    Trains a log reg for a certain scale 
    """
    logreg_table = []
    size = int(100/scale[0]*scale[1])
    
    train_accuracy, test_accuracy = np.zeros(size), np.zeros(size)

    # loop over regularisation strength
    for i, pixel in tqdm(enumerate(np.transpose(Y_train))):
    
        # define logistic regressor
        logreg = linear_model.LogisticRegression(penalty="l2", C=0.05, max_iter = 15000, multi_class='multinomial')#, n_jobs = 2)#, n_jobs=-1)
    
        # fit training data
        logreg.fit(X_train, pixel)
    
        logreg_table.append(logreg)
    
        # check accuracy
        train_accuracy[i]=logreg.score(X_train,pixel)
        test_accuracy[i]=logreg.score(X_test,np.transpose(Y_test)[i])
        
    return logreg_table, train_accuracy, test_accuracy

In [ ]:
# Tried a linear regression but results are shit lol

def train_linreg_scale(X_train, Y_train, X_test, Y_test, scale) :
    """
    Trains a lin reg for a certain scale 
    """
    linreg_table = []
    size = int(100/scale[0]*scale[1])
    
    train_accuracy, test_accuracy = np.zeros(size), np.zeros(size)

    # loop over regularisation strength
    for i, pixel in tqdm(enumerate(np.transpose(Y_train))):
    
        # define linistic regressor
        linreg = linear_model.LinearRegression()#, n_jobs = 2)#, n_jobs=-1)
    
        # fit training data
        linreg.fit(X_train, pixel)
    
        linreg_table.append(linreg)
    
        # check accuracy
        train_accuracy[i]=linreg.score(X_train,pixel)
        test_accuracy[i]=linreg.score(X_test,np.transpose(Y_test)[i])
        
    return linreg_table, train_accuracy, test_accuracy

In [35]:
#Prediction functions for initial pixels
[logreg_table_11, train_accuracy_11, test_accuracy_11] = train_logreg_scale(X_train, Y_train, X_test, Y_test, (1,1))

100it [22:16, 13.37s/it]


In [36]:
print(np.average(train_accuracy_11), np.average(test_accuracy_11))

1.0 0.8508216783216782


In [ ]:
# New Y_train for 2*2 block averaged images
Y_train_22, Y_test_22 = Y_train_newscale(Y_train, pixel_indices_22), Y_train_newscale(Y_test, pixel_indices_22)

[logreg_table_22, train_accuracy_22, test_accuracy_22] = train_logreg_scale(X_train, Y_train_22, X_test, Y_test_22,(2,2))

44it [23:55, 50.89s/it]

In [ ]:
print(np.average(train_accuracy_22), np.average(test_accuracy_22))

In [14]:
# New Y_train for 2*2 block averaged images
Y_train_22, Y_test_22 = Y_train_newscale(Y_train, pixel_indices_22), Y_train_newscale(Y_test, pixel_indices_22)

# Linear regression 
[linreg_table_22, train_accuracy_22, test_accuracy_22] = train_linreg_scale(X_train, Y_train_22, X_test, Y_test_22,(2,2))

25it [01:08,  2.75s/it]


In [15]:
print(np.average(train_accuracy_22), np.average(test_accuracy_22))

0.25 0.020929925825320245


In [14]:
# New Y_train for 2*1 block averaged images
Y_train_21, Y_test_21 = Y_train_newscale(Y_train, pixel_indices_21),  Y_train_newscale(Y_test, pixel_indices_21)
            
[logreg_table_21, train_accuracy_21, test_accuracy_21] = train_logreg_scale(X_train, Y_train_21, X_test, Y_test_21,(2,1))

25it [17:36, 42.25s/it]


KeyboardInterrupt: 

In [15]:
print(np.average(train_accuracy_21), np.average(test_accuracy_21))

NameError: name 'train_accuracy_21' is not defined

In [16]:
# New Y_train for 1*2 block averaged images
Y_train_12, Y_test_12 = Y_train_newscale(Y_train, pixel_indices_12),  Y_train_newscale(Y_test, pixel_indices_12)

[logreg_table_12, train_accuracy_12, test_accuracy_12] = train_logreg_scale(X_train, Y_train_12, X_test, Y_test_12,(1,2))

0it [00:03, ?it/s]


KeyboardInterrupt: 

In [ ]:
print(np.average(train_accuracy_12), np.average(test_accuracy_12))
len(Y_train_12[0])

# Check results 

In [34]:
# Compute predictions on geometric image n°n for 2x2 and 1x1

def test_predictions(n) :
    """
    Just a function that returns the predictions for 2x2 block averaged and no averaging of the n-th geometric
    image. Only useful to check some plots.
    """
    imag = stimuli_figures2[n]

    ytest_22 = preproc_one_stimuli(pixel_indices_22, imag, reshape = True)
    ytest_11 = np.reshape(imag,(10,10))
    ytest_12 = preproc_one_stimuli(pixel_indices_12, imag, reshape = True, reshape_form = (5,10))
    ytest_21 = preproc_one_stimuli(pixel_indices_21, imag, reshape = True, reshape_form = (10,5))
    xtest = fmri_figures_data2[n].reshape(1,-1)

    ypred_22 = np.zeros(25)
    ypred_11 = np.zeros(100)
    ypred_21 = np.zeros(50)
    ypred_12 = np.zeros(50)

    for i,pixel in enumerate(ypred_22) :
        y22 = logreg_table_22[i].predict(xtest)
        ypred_22[i] = y22
    ypred_22_greyscale = [x/4 for x in ypred_22]
    ypred_22_greyscale = np.reshape(ypred_22_greyscale,(5,5))
    
    for i,pixel in enumerate(ypred_12) :
        y12 = logreg_table_12[i].predict(xtest)
        ypred_12[i] = y12
    ypred_12_greyscale = [x/2 for x in ypred_12]
    ypred_12_greyscale = np.reshape(ypred_12_greyscale,(5,10))
    
    for i,pixel in enumerate(ypred_21) :
        y21 = logreg_table_21[i].predict(xtest)
        ypred_21[i] = y21
    ypred_21_greyscale = [x/4 for x in ypred_21]
    ypred_21_greyscale = np.reshape(ypred_21_greyscale,(10,5))

    for i, pixel in enumerate(ypred_11) :
        y11 = logreg_table_11[i].predict(xtest)
        ypred_11[i] = y11
    ypred_11 = np.reshape(ypred_11,(10,10))
    
    return ytest_11, ypred_11,  ytest_22, ypred_22_greyscale, ytest_21, ypred_21_greyscale, ytest_12, ypred_12_greyscale 

In [ ]:
# Check prediction and true value for 2x2 and no block averaging : ok
ytest_11, ypred_11,  ytest_22, ypred_22_greyscale, ytest_21, ypred_21_greyscale, ytest_12, ypred_12_greyscale  = test_predictions(25)

fig = plt.figure(figsize=(6,6))
sp1 = plt.subplot(421)
sp2 = plt.subplot(422)
sp3 = plt.subplot(423)
sp4 = plt.subplot(424)
sp5 = plt.subplot(425)
sp6 = plt.subplot(426)
sp7 = plt.subplot(427)
sp8 = plt.subplot(428)
sp1.imshow(ytest_22, cmap = plt.cm.gray, interpolation = 'nearest', vmin=0, vmax = 1)
sp2.imshow(ypred_22_greyscale, cmap = plt.cm.gray, interpolation = 'nearest',vmin=0, vmax = 1)
sp3.imshow(ytest_11, cmap = plt.cm.gray, interpolation = 'nearest', vmin=0, vmax = 1)
sp4.imshow(ypred_11, cmap = plt.cm.gray, interpolation = 'nearest', vmin=0, vmax = 1)
sp5.imshow(ytest_12, cmap = plt.cm.gray, interpolation = 'nearest', vmin=0, vmax = 1, aspect = 2)
sp6.imshow(ypred_12_greyscale, cmap = plt.cm.gray, interpolation = 'nearest', vmin=0, vmax = 1, aspect = 2)
sp7.imshow(ytest_21, cmap = plt.cm.gray, interpolation = 'nearest', vmin=0, vmax = 1, aspect = 0.5)
sp8.imshow(ypred_21_greyscale, cmap = plt.cm.gray, interpolation = 'nearest', vmin=0, vmax = 1, aspect = 0.5)
print("Accuracy 1x1 : ", np.average(test_accuracy_11), "\nAccuracy 2x2 :", np.average(test_accuracy_22))

# Let's try to merge the two results of 2x2 and 1x1

Since the 2x2 block averaged predictions have 25 pixels, we have to duplicate the pixels into 4 pixels to get a 10x10 image (and not a 5x5). That way, the 1x1 and 2x2 block-avg images will have the same format and we will be able to do a linear combination of their greyscale values. 

Idea : maybe, after this manipulation (getting 10x10 pixels for every prediction), we can check what pixels are best predicted by what model (2x2 or 1x1) and adapt weights. We could also try different linear combinations.

In [ ]:
# still to add the reshaping for 2*1 and 1*2 images

def reshape_to_100_pixels(img, shape = (5,5)) :
    """
    Reshapes a 5*5 image into a 10*10 image by duplicating pixels
    Not really opti sorry
    - shape = original shape of image
    - img = img to reshape to 10*10. Can be given as a flat list bc
            we reshape it in the beggining of the function.
    """
    img = np.array(np.reshape(img,shape)) #for use of indicies
    img100 = np.zeros((10,10))
    
    if shape == (5,5) :
        for i in range(len(img[0])) :
            for j in range(len(img[0])) :
                img100[2*i, 2*j] = img[i,j]
                img100[2*i,2*j+1] = img[i,j]
                img100[2*i+1,2*j] = img[i,j]
                img100[2*i+1,2*j+1] = img[i,j]
                
    #elif shape == (1,2) :
      #  for 
            
    return img100


        

In [ ]:
default = np.zeros(100)
def avg_image(img11, img22, w11, w22 = default, w12 = default, w21 = default, reshaped = False) :
    """
    Averages values of pixels given the matrices of weights (1 weight per pixel per type of block averaging)
    Put reshaped = True if you want the output to be a 10*10 array and not a flat 100-elements long list
    """
    
    # Flatten in needed
    img11.flatten()
    img22.flatten()
    
    # Check sum of weights is one
    for i in range(len(w11)) :
        if np.any(w11 + w22) != 1 :
            print("Sum of weights is not one, see pixel no", i)
            break
    
    # Averaged image
    merged = np.array([img11[i]*w11[i] + img22[i]*w22[i] for i in range(len(img11))])
    
    # Reshapes to 10*10 format if specified in arguments of the function
    if reshaped == False :
        return merged
    else :
        return np.reshape(merged,(10,10))
    


In [ ]:
#Pas eu le temps de tester mais à moins d'être très con je pense que ça marche

#reshape all 
def reshape_all_images(img_array, shape = (5,5)):
    """
    Takes the initial stimuli and reshapes them all in the same fashion
    """
    
    reshaped_array = np.zeros(np.shape(img_array[0]))
    print("Reshaping...")
    for i in tqdm(range(np.shape(img_array)[0])):
        reshaped_array[i] = reshape_to_100_pixels(img_array[i], shape = (5,5))
    return reshaped_array

def avg_all_images(img_array_11, img_array_22, #img_array_12, img_array_21, 
                  w11, w22 = default, w12 = default, w21 = default, reshaped = False):
    
    #N images
    averages_array = np.zeros(np.shape(img_array_11)[0])
    print("Averaging...")
    for i in tqdm(range(np.shape(img_array_11)[0])):
        averages_array[i] = avg_image(img_array_11[i], img_array_22[i], #img_array_12[i], img_array_21[i], 
                                      w11, w22 = default, w12 = default, w21 = default, reshaped = False)
    return averages_array
    

In [ ]:
# Ok it works (average img 2x2 and 1x1)
fig = plt.figure()
sp1 = plt.subplot(151)
sp2 = plt.subplot(152)
sp3 = plt.subplot(153)
sp4 = plt.subplot(154)
sp5 = plt.subplot(155)

w11 = 0.5 * np.ones(100)
w22 = 0.5 * np.ones(100)

n=88
ytest_11, ypred_11,  ytest_22, ypred_22_greyscale, ytest_21, ypred_21_greyscale, ytest_12, ypred_12_greyscale  = test_predictions(n)

sp1.imshow(ypred_11, cmap = plt.cm.gray, interpolation = 'nearest', vmin=0, vmax=1)
sp2.imshow(ypred_22_greyscale, cmap = plt.cm.gray, interpolation = 'nearest', vmin=0, vmax=1)

ypred_22_100 = reshape_to_100_pixels(ypred_22_greyscale)
newimg = avg_image(img11 = ypred_11, img22 = ypred_22_100, w11 = w11, w22 = w22)
newimg_bin = [[round(val,0) for val in newimg[i]] for i in range(len(newimg))]

sp3.imshow(newimg, cmap = plt.cm.gray, vmin = 0, vmax = 1)

sp4.imshow(ytest_11, cmap = plt.cm.gray, vmin = 0, vmax = 1) #real value
sp5.imshow(newimg_bin, cmap = plt.cm.gray, vmin = 0, vmax = 1) #binarized avg pred

This works : now we want also to be able to do it with the 2x1 and 1x2 block averaged images. But before, let's maybe look into the optimization ?

We should try to find which pixels are best predicted by each block averaging.

Also we need better trained models on the 2x2 and 2x1 or 1x2 i don't remember which one is the worst.

In [25]:
#Optimization of the logregs
#Idea: my laptop is too slow to optimize on all pixels; maybe optimize on a few and see if always gives the same
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn import linear_model
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score

X_train = fmri_data
y_train = [x[0] for x in stimuli]

space = {

    'warm_start' : hp.choice('warm_start', [True, False]),
    'fit_intercept' : hp.choice('fit_intercept', [True, False]),
    'tol' : hp.choice('tol', [0.00001, 0.0001]),
    'C' : hp.choice('C', [0.05, 0.075])
    #'solver' : hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'])
}



def objective(params, n_folds = 1):
    """Objective function for Logistic Regression Hyperparameter Tuning"""

    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evaluate based on ROC AUC

    clf = linear_model.LogisticRegression(**params,random_state=0,verbose =2, solver='lbfgs', max_iter = 15000)
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    score = cross_val_score(clf, X_train, y_train, cv=cv, scoring='f1_macro')

    # Extract the best score
    best_score = max(score)

    # Loss must be minimized
    loss = 1 - best_score

    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

trials=Trials()
best = fmin(objective,
            space, 
            algo=tpe.suggest, 
            trials=trials, 
            max_evals=50
            )

print ("Best result:", best)
print (trials.best_trial)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.8s remaining:    0.0s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.8s finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.0s remaining:    0.0s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.0s finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.1s remaining:    0.0s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.1s finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.8s remaining:    0.0s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.8s finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 con

  2%|▏         | 1/50 [02:18<1:53:23, 138.84s/trial, best loss: 0.5946673189823874]

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.3s remaining:    0.0s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.3s finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.8s remaining:    0.0s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.8s finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.7s remaining:    0.0s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.7s finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.2s remaining:    0.0s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.2s finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.7s rem

  4%|▍         | 2/50 [08:27<3:39:06, 273.88s/trial, best loss: 0.23315217391304344]

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.1s remaining:    0.0s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.1s finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.3s remaining:    0.0s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.3s finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.9s remaining:    0.0s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.9s finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.3s remaining:    0.0s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.3s finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.7s rem

 98%|█████████▊| 49/50 [3:47:52<04:25, 265.72s/trial, best loss: 0.23315217391304344]

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.9s remaining:    0.0s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.9s finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.8s remaining:    0.0s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.8s finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.7s remaining:    0.0s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.7s finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.2s remaining:    0.0s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.2s finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.5s rem

100%|██████████| 50/50 [3:53:27<00:00, 280.14s/trial, best loss: 0.23315217391304344]
Best result: {'C': 1, 'fit_intercept': 0, 'tol': 1, 'warm_start': 0}
{'state': 2, 'tid': 1, 'spec': None, 'result': {'loss': 0.23315217391304344, 'params': {'C': 0.075, 'fit_intercept': True, 'tol': 0.0001, 'warm_start': True}, 'status': 'ok'}, 'misc': {'tid': 1, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'C': [1], 'fit_intercept': [1], 'tol': [1], 'warm_start': [1]}, 'vals': {'C': [1], 'fit_intercept': [0], 'tol': [1], 'warm_start': [0]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2022, 3, 7, 21, 4, 44, 317000), 'refresh_time': datetime.datetime(2022, 3, 7, 21, 10, 52, 720000)}


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.3s remaining:    0.0s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.3s finished



In [26]:
#Optimization of the logregs
#Idea: my laptop is too slow to optimize on all pixels; maybe optimize on a few and see if always gives the same
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn import linear_model
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score

X_train = fmri_data
y_train = [x[0] for x in stimuli]

space = {

    'warm_start' : hp.choice('warm_start', [True, False]),
    'fit_intercept' : hp.choice('fit_intercept', [True, False]),
    'tol' : hp.choice('tol', [0.00001, 0.0001]),
    'C' : hp.choice('C', [0.025,0.05, 0.075, 0.1]),
    'solver' : hp.choice('solver', ['liblinear','lbfgs', 'sag'])
}



def objective(params, n_folds = 1):
    """Objective function for Logistic Regression Hyperparameter Tuning"""

    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evaluate based on ROC AUC

    clf = linear_model.LogisticRegression(**params,random_state=0,verbose =3, max_iter = 15000)
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    score = cross_val_score(clf, X_train, y_train, cv=cv, scoring='f1_macro')

    # Extract the best score
    best_score = max(score)

    # Loss must be minimized
    loss = 1 - best_score

    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

trials=Trials()
best = fmin(objective,
            space, 
            algo=tpe.suggest, 
            trials=trials, 
            max_evals=50
            )

print ("Best result:", best)
print (trials.best_trial)

SyntaxError: invalid syntax (<ipython-input-26-1bf2ffe64919>, line 16)